In [80]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3226227024945303117, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 133627904
 locality {
   bus_id: 1
 }
 incarnation: 6493536985566421878
 physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:02:00.0, compute capability: 6.1"]

## Start programming!!

### import modules

In [81]:
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from keras import layers
from keras.models import Model, load_model
from keras import regularizers
import keras

from sklearn.metrics import accuracy_score

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Dense, GlobalAveragePooling2D, Lambda
from keras.layers import Conv2D, MaxPooling2D
import os

### load data


In [82]:
folder = '/home/baiyeqi/Data Science/Data/Sign Language MNIST/Processed Data/v1'
X_train = np.load(folder + '/X_train.npy')
X_test = np.load(folder + '/X_test.npy')
y_train = np.load(folder + '/Y_train.npy')
y_test = np.load(folder + '/Y_test.npy')

In [83]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4691, 28, 28), (1088, 28, 28), (4691,), (1088,))

In [84]:
#one hot encoding
encoder = OneHotEncoder()
encoder = encoder.fit(y_test.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1)).toarray()

encoder = OneHotEncoder()
encoder = encoder.fit(y_train.reshape(-1, 1))
y_train = encoder.transform(y_train.reshape(-1, 1)).toarray()


In [85]:
x_train = X_train.astype('float32')
x_test = X_test.astype('float32')
x_train /= 255
x_test /= 255
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))

In [86]:
x_test.shape

(1088, 28, 28, 1)

### Build the model

In [87]:
import gc
gc.collect()

5997

In [88]:
# softmax for tf version

def soft_max(x):
    import tensorflow as tf
    return tf.nn.softmax(x)

In [89]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))

In [90]:
dense_out = model.output

predictions = Lambda(lambda x: soft_max(x))(dense_out)

# this is the model we will train
model = Model(inputs=model.input, outputs=predictions)

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')


### Prediction

In [91]:
def predict_score(model=model):
    
    y_pred = model.predict(x_test)

    y_pred = np.round(y_pred)

    score = accuracy_score(y_test, y_pred)

    return score



In [92]:
epoch = 2
for i in range(epoch):
    model.fit(x = x_train, y = y_train, validation_data=(x_test, y_test), epochs = 1, batch_size = 16)
    score = predict_score(model)
    print("Epoch " + str(i) + ": accuracy = " + str(score))

Train on 4691 samples, validate on 1088 samples
Epoch 1/1
4691/4691 [==============================] - 2s 478us/step - loss: 0.2918 - val_loss: 0.1526
Epoch 0: accuracy = 0.9503676470588235
Train on 4691 samples, validate on 1088 samples
Epoch 1/1
4691/4691 [==============================] - 2s 426us/step - loss: 0.0203 - val_loss: 0.0437
Epoch 1: accuracy = 0.9816176470588235


## Save the model without softmax

In [94]:
new_model = Model(inputs=model.input, outputs=dense_out)
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26_input (InputLayer) (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
activation_32 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 26, 26, 32)        9248      
_________________________________________________________________
activation_33 (Activation)   (None, 26, 26, 32)        0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 13, 13, 32)        0         
__________

In [95]:
new_model.save('sign lang v1.h5')

## Reload Model

In [21]:
from keras.models import load_model

In [22]:
lmodel = load_model('/home/baiyeqi/Data Science/Models/Brain Cells/3 level/acc_9_75_base.h5')

/home/baiyeqi/anaconda3/envs/TensorFlow/lib/python3.5/site-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


### Put back the softmax layer

In [26]:
# softmax for tf version

def soft_max(x):
    import tensorflow as tf
    return tf.nn.softmax(x)

In [27]:
from keras import optimizers
rms = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

In [28]:
x = lmodel.output
predictions = Lambda(lambda x: soft_max(x))(x)
lmodel = Model(inputs=lmodel.input, outputs=predictions)
lmodel.compile(optimizer=rms, loss='categorical_crossentropy')

In [ ]:
epoch = 1
for i in range(epoch):
    model.fit(x = X_train, y = y_train, validation_data=(X_test, y_test), epochs = 1, batch_size = 8)
    score = predict_score()
    print("Epoch " + str(i) + ": accuracy = " + str(score))
    # model.save("drive/Colab Notebooks/Shopee Competition/Tops/Models/Tops Inception/Tops_Inception_epoch" + str(i) + ".h5")

Train on 8100 samples, validate on 900 samples
Epoch 1/1
